<a href="https://colab.research.google.com/github/eduardez/Machine-Learning-ESI/blob/master/Dengue_Task2_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries

In [1]:
%matplotlib inline
from google.colab import drive

from __future__ import print_function
from __future__ import division

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import statsmodels.api as sm

# just for the sake of this blog post!
from warnings import filterwarnings
filterwarnings('ignore')

#Autocorrelation
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# load the provided data
train_features_raw = pd.read_csv('https://raw.githubusercontent.com/eduardez/Machine-Learning-ESI/master/dengue_features_train.csv?token=AGI3DY2BIJEZHXH6MWD42ZS77LZTE',
                             index_col=[0,1,2])

train_labels_raw = pd.read_csv('https://raw.githubusercontent.com/eduardez/Machine-Learning-ESI/master/dengue_labels_train.csv?token=AGI3DY6W2UZTEKJAKUJFXQC77LZVQ',
                             index_col=[0,1,2])

features_test_raw = pd.read_csv('https://raw.githubusercontent.com/eduardez/Machine-Learning-ESI/master/dengue_features_test.csv?token=AGI3DYYWLT5OHQAQYTJXKPK77LZP2',
                             index_col=[0,1,2])

submission = pd.read_csv('https://raw.githubusercontent.com/eduardez/Machine-Learning-ESI/master/submission_format.csv?token=AGI3DY74P5TFADKZGDSLOZS77LZWM',
                             index_col=[0,1,2])

# Data gathering

In [3]:
train_features = train_features_raw.copy()
train_labels = train_labels_raw.copy()

In [4]:
# Seperate data for San Juan
sj_train_features = train_features.loc['sj']
sj_train_labels = train_labels.loc['sj']

# Separate data for Iquitos
iq_train_features = train_features.loc['iq']
iq_train_labels = train_labels.loc['iq']

In [5]:
sj_train_features.fillna(method='ffill', inplace=True)
iq_train_features.fillna(method='ffill', inplace=True)

# Data shift

In [6]:
def cross_corr(x, y, lag=0):
    return y.corr(x.shift(lag))


def shift_Factor(x, y, R):
    x_corr = [cross_corr(x, y, lag=i) for i in range(R)]

    # R factor is the number of the shifts who should be checked by the function
    Kot = pd.DataFrame(list(x_corr)).reset_index()
    Kot.rename(columns={0: 'Corr', 'index': 'Shift_num'}, inplace=True)

    # We find optimal correlation shift
    Kot['abs'] = Kot['Corr'].abs()
    SF = Kot.loc[Kot['abs'] == Kot['abs'].max(), 'Shift_num']
    p1 = SF.to_frame()
    SF = p1.Shift_num.max()

    return SF
#for generic dataset
def df_shif(df, target=None, lag=0):
    if not lag and not target:
        return df       
    new = {}
    for h in df.columns:
        if h == target:
            new[h] = df[target]
        else:
            new[h] = df[h].shift(periods=lag, fill_value = 0)
    return  pd.DataFrame(data=new)

#for specific colum
def df_shif_column(df, column, lag=0, replace=False):
  if replace:
    df[column] = df[column].shift(periods=lag, fill_value = 0)
  if not replace:
    if lag == 0:
      df[column] = df[column].shift(periods=lag, fill_value = 0)
    else:
      df[column+'_'+str(lag)+'_weeks'] = df[column].shift(periods = lag, fill_value=0)

## San Juan

In [7]:
#SJ dictionary
sj_dic = {
    'ndvi_ne': 9,
    'ndvi_nw': 3,
    'ndvi_se' : 0,
    'ndvi_sw' : 0,
    'precipitation_amt_mm':0,
    'reanalysis_air_temp_k' :0,
    'reanalysis_avg_temp_k':0,
    'reanalysis_dew_point_temp_k':0,
    'reanalysis_max_air_temp_k':0,
    'reanalysis_min_air_temp_k':0,
    'reanalysis_precip_amt_kg_per_m2':0,
    'reanalysis_relative_humidity_percent':0,
    'reanalysis_sat_precip_amt_mm':0,
    'reanalysis_specific_humidity_g_per_kg':0,
    'reanalysis_tdtr_k':0,
    'station_avg_temp_c':0,
    'station_diur_temp_rng_c':0,
    'station_max_temp_c':0,
    'station_min_temp_c':0,
    'station_precip_mm': 0
}
#Change boolean for replace or not
for item in sj_dic:
  df_shif_column(sj_train_features, item, sj_dic[item], False)

sj_train_features

week_start_date   ndvi_ne  ...  ndvi_ne_9_weeks  ndvi_nw_3_weeks
year weekofyear                            ...                                  
1990 18              1990-04-30  0.122600  ...          0.00000         0.000000
     19              1990-05-07  0.169900  ...          0.00000         0.000000
     20              1990-05-14  0.032250  ...          0.00000         0.000000
     21              1990-05-21  0.128633  ...          0.00000         0.103725
     22              1990-05-28  0.196200  ...          0.00000         0.142175
...                         ...       ...  ...              ...              ...
2008 13              2008-03-25  0.077850  ...         -0.02680        -0.090033
     14              2008-04-01 -0.038000  ...         -0.02680        -0.141200
     15              2008-04-08 -0.155200  ...         -0.11170         0.024450
     16              2008-04-15  0.001800  ...          0.07200        -0.039900
     17              2008-04-22 -0.037000  ...         -0.13865        -0.016833

[936 rows x 23 columns]

## Iquitos

In [8]:
#Iq dictionary
iq_dic = {
    'ndvi_ne': 9,
    'ndvi_nw': 3,
    'ndvi_se' : 0,
    'ndvi_sw' : 0,
    'precipitation_amt_mm':0,
    'reanalysis_air_temp_k' :0,
    'reanalysis_avg_temp_k':0,
    'reanalysis_dew_point_temp_k':0,
    'reanalysis_max_air_temp_k':0,
    'reanalysis_min_air_temp_k':0,
    'reanalysis_precip_amt_kg_per_m2':0,
    'reanalysis_relative_humidity_percent':0,
    'reanalysis_sat_precip_amt_mm':0,
    'reanalysis_specific_humidity_g_per_kg':0,
    'reanalysis_tdtr_k':0,
    'station_avg_temp_c':0,
    'station_diur_temp_rng_c':0,
    'station_max_temp_c':0,
    'station_min_temp_c':0,
    'station_precip_mm': 0
}
#Change boolean for replace or not
for item in sj_dic:
  df_shif_column(iq_train_features, item, iq_dic[item], False)

sj_train_features

week_start_date   ndvi_ne  ...  ndvi_ne_9_weeks  ndvi_nw_3_weeks
year weekofyear                            ...                                  
1990 18              1990-04-30  0.122600  ...          0.00000         0.000000
     19              1990-05-07  0.169900  ...          0.00000         0.000000
     20              1990-05-14  0.032250  ...          0.00000         0.000000
     21              1990-05-21  0.128633  ...          0.00000         0.103725
     22              1990-05-28  0.196200  ...          0.00000         0.142175
...                         ...       ...  ...              ...              ...
2008 13              2008-03-25  0.077850  ...         -0.02680        -0.090033
     14              2008-04-01 -0.038000  ...         -0.02680        -0.141200
     15              2008-04-08 -0.155200  ...         -0.11170         0.024450
     16              2008-04-15  0.001800  ...          0.07200        -0.039900
     17              2008-04-22 -0.037000  ...         -0.13865        -0.016833

[936 rows x 23 columns]

## Autoshift

In [9]:
def autoshift(df, output):
  for i in df.columns[1:]:
    x = df[i]
    SKO = shift_Factor(x,y,R)
    print('Optimal shift for ' + i + ' : ', SKO)
    print(cross_corr(x, y, lag=SKO))
    output = output.append(df_shif(df, i, lag=SKO))

In [10]:
y = sj_train_labels.total_cases #target
R = 14 #week of maximun shift
sj_train_features_shift = pd.DataFrame() #df from sj with autoshift
iq_train_features_shift = pd.DataFrame() #df from iq with autoshift

autoshift(sj_train_features, sj_train_features_shift)

Optimal shift for ndvi_ne :  13
0.06342923840238575
Optimal shift for ndvi_nw :  11
0.1090804456404502
Optimal shift for ndvi_se :  2
-0.12293545697099086
Optimal shift for ndvi_sw :  1
0.0468981735748321
Optimal shift for precipitation_amt_mm :  2
0.08026087637645954
Optimal shift for reanalysis_air_temp_k :  8
0.2974226163184767
Optimal shift for reanalysis_avg_temp_k :  8
0.29434132774241956
Optimal shift for reanalysis_dew_point_temp_k :  8
0.3012885220820066
Optimal shift for reanalysis_max_air_temp_k :  7
0.29561179016553346
Optimal shift for reanalysis_min_air_temp_k :  8
0.3025964001546403
Optimal shift for reanalysis_precip_amt_kg_per_m2 :  2
0.1232217514468022
Optimal shift for reanalysis_relative_humidity_percent :  3
0.16480748725687427
Optimal shift for reanalysis_sat_precip_amt_mm :  2
0.08026087637645954
Optimal shift for reanalysis_specific_humidity_g_per_kg :  8
0.3010254971093241
Optimal shift for reanalysis_tdtr_k :  2
-0.08860594521794571
Optimal shift for station_a

In [11]:
autoshift(iq_train_features,iq_train_features_shift)

Optimal shift for ndvi_ne :  0
0.1812490426818961
Optimal shift for ndvi_nw :  2
0.20967651847908622
Optimal shift for ndvi_se :  2
0.18327704478026471
Optimal shift for ndvi_sw :  0
0.2567151913609842
Optimal shift for precipitation_amt_mm :  6
-0.2669111224087361
Optimal shift for reanalysis_air_temp_k :  0
0.32006777560200317
Optimal shift for reanalysis_avg_temp_k :  0
0.31883515795645373
Optimal shift for reanalysis_dew_point_temp_k :  8
-0.3681277055458622
Optimal shift for reanalysis_max_air_temp_k :  0
0.3755878377303714
Optimal shift for reanalysis_min_air_temp_k :  8
-0.3369297809080478
Optimal shift for reanalysis_precip_amt_kg_per_m2 :  6
-0.2534490161431323
Optimal shift for reanalysis_relative_humidity_percent :  3
-0.3787972841341112
Optimal shift for reanalysis_sat_precip_amt_mm :  6
-0.2669111224087361
Optimal shift for reanalysis_specific_humidity_g_per_kg :  8
-0.37422291671036695
Optimal shift for reanalysis_tdtr_k :  3
0.37532655119987446
Optimal shift for station_